In [192]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
#from ydata-profiling import Repo

In [211]:
url = "https://www.columbiasportswear.fr/FR/c/men-featured-bestsellers"
head = {"User-Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0"}
r = requests.get(url, headers  = head)
print(r.status_code)



soup = BeautifulSoup(r.text, "html.parser")
links = soup.find_all("a",  {"class" : "item-link"})

items_links_women = []
for link in links:
    href = link.get('href')
    items_links_women.append(href)



url = "https://www2.hm.com/fr_fr/homme/nouveautes/view-all.html?sort=stock&productTypes=Casquette,Chapeau,Chaussures,Chemise,Cravate,Jean,Legging,Pantalon,Pantalon+de+pyjama,Pull,Pyjama,Short,Short+de+bain,Short+de+pyjama,Sweat,T-shirt,Top,Veste&image-size=small&image=model&offset=0&page-size=443"
head = {"User-Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0"}
r = requests.get(url, headers  = head)
print(r.status_code)

soup = BeautifulSoup(r.text, "html.parser")
links = soup.find_all("a",  {"class" : "item-link"})



items_links_men = []
for link in links:
    href = link.get('href')
    items_links_men.append(href)

list_items = items_links_women + items_links_men

200
200


In [212]:
print(list_items.index("/fr_fr/productpage.1129645011.html"))

412


In [213]:
list_items.remove("/fr_fr/productpage.1129645011.html")

In [214]:
# Initialize lists to store the items details
cloths = []
color = []
gender = []
product_id_cloth = []
price = []
company = []
factory = []
employee = []
    
    
# Initialize lists to store the composition details
article_number_composition = []
layer = []
material = []
percentage = []



In [216]:
for item_url in list_items:
    print(list_items.index(item_url))
    url = f"https://www2.hm.com{item_url}"
    print(url)
    head = {"User-Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0"}
    r = requests.get(url, headers  = head, timeout = None)
    
    
     # Parse the HTML content
    soup = BeautifulSoup(r.text, "html.parser")
    
    
    
    title = soup.find("title").text
    color.append(title.split(" - ")[1])
    cloths.append(title.split(" ")[0])
    gender.append(title.split(" - ")[2].split(" | ")[0])
    price.append(soup.find("span",{"class": "price-value"}).text)
   

    # Find the div element containing the composition section
    details_div = soup.find("div", {"class": "content pdp-text pdp-content"})
    

    # Find the div element containing the   product ID
    product_id_div = details_div.find_all("dd")[-1]

    # Extract the product ID
    product_id_cloth.append(product_id_div.text)
    
    # Find the dl element containing the composition information
    composition = details_div.find_all("li")
    composition.pop(0)
    

    # Loop over the composition list
    for item in composition:
        # Remove the <li> and </li> tags from the item string
        item = item.text.strip()
        print(item)
        
        if ":" in item: 
            # Split the item string into its layer and material/percentage components
            layer_material, materials = item.split(":")
    
            # Loop over the material strings
            for material_str in materials.split(", "):
                # Split the material string into its material and percentage components
                material_percentage = material_str.split(" ")[-1]
                material_str = " ".join(material_str.split(" ")[:-1]).lstrip()
                percentage_float = float(material_percentage.strip("%"))
        
                # Append the details to the lists
                article_number_composition.append(product_id)
                layer.append(layer_material)
                material.append(material_str)
                percentage.append(percentage_float)
                
        else:
            # Loop over the material strings
            for material_str in materials.split(", "):
                # Split the material string into its material and percentage components
                material_percentage = material_str.split(" ")[-1]
                material_str = " ".join(material_str.split(" ")[:-1]).lstrip()
                percentage_float = float(material_percentage.strip("%"))
        
                # Append the details to the lists
                article_number_composition.append(product_id)
                layer.append(np.nan)
                material.append(material_str)
                percentage.append(percentage_float)

    
    list_items.remove(item_url)      




0
https://www2.hm.com/fr_fr/productpage.1166969003.html
None


AttributeError: 'NoneType' object has no attribute 'find'

In [207]:
# Create the dataframe
df_tendance_composition = pd.DataFrame({
"product_id": article_number_composition,
"layer": layer,
"material": material,
"percentage": percentage
})

df_tendance_cloth = pd.DataFrame({
    "product_id" : product_id_cloth,
    "gender" : gender,
    "clothes" : cloths,
    "color" : color,
    "price" : price
})

In [217]:
df_tendance_composition.to_csv("data_h&m_tendance_compo.csv")

In [218]:
df_tendance_cloth.to_csv("data_h&m_tendnace_clothes.csv")

In [228]:
list_id = df_tendance_cloth["product_id"].to_list()

all_rows = []

for product_id in list_id:
    print(list_id.index(product_id))
    url = f"https://www2.hm.com/fr_fr/supplierDetails/articles/{product_id}"
    print(url)
    head = {"User-Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0"}
    r = requests.get(url, headers  = head, timeout = None)

    # Vérifier si la requête a réussi
    if r.status_code == 200:
        response_json = r.json()

        # Extraire les données pertinentes
        for country in response_json["countries"]:
            country_name = country["name"]

            for supplier in country["suppliers"]:
                supplier_name = supplier["name"]

                for factory in supplier["factories"]:
                    factory_name = factory["name"]
                    workers_number = factory.get("workersNumber", "N/A")

                    all_rows.append({
                        "Product ID": product_id,
                        "Country": country_name,
                        "Company": supplier_name,
                        "Factory": factory_name,
                        "Number of Employees": workers_number
                    })

    else:
        print(f"Error: Request failed with status code {r.status_code} for product ID: {product_id}")

0
https://www2.hm.com/fr_fr/supplierDetails/articles/1166969003
1
https://www2.hm.com/fr_fr/supplierDetails/articles/1142215002
2
https://www2.hm.com/fr_fr/supplierDetails/articles/1159122003
3
https://www2.hm.com/fr_fr/supplierDetails/articles/1159122001
4
https://www2.hm.com/fr_fr/supplierDetails/articles/1157667001
5
https://www2.hm.com/fr_fr/supplierDetails/articles/1158221001
6
https://www2.hm.com/fr_fr/supplierDetails/articles/1156096001
7
https://www2.hm.com/fr_fr/supplierDetails/articles/1159124002
8
https://www2.hm.com/fr_fr/supplierDetails/articles/1159502001
9
https://www2.hm.com/fr_fr/supplierDetails/articles/1180370002
10
https://www2.hm.com/fr_fr/supplierDetails/articles/1139990003
11
https://www2.hm.com/fr_fr/supplierDetails/articles/1157505004
12
https://www2.hm.com/fr_fr/supplierDetails/articles/1157505003
13
https://www2.hm.com/fr_fr/supplierDetails/articles/1121055005
14
https://www2.hm.com/fr_fr/supplierDetails/articles/1150536002
15
https://www2.hm.com/fr_fr/suppli

125
https://www2.hm.com/fr_fr/supplierDetails/articles/1145299001
126
https://www2.hm.com/fr_fr/supplierDetails/articles/1169732001
127
https://www2.hm.com/fr_fr/supplierDetails/articles/1129879003
128
https://www2.hm.com/fr_fr/supplierDetails/articles/1145276001
129
https://www2.hm.com/fr_fr/supplierDetails/articles/1152722002
130
https://www2.hm.com/fr_fr/supplierDetails/articles/1153730002
131
https://www2.hm.com/fr_fr/supplierDetails/articles/1066926004
132
https://www2.hm.com/fr_fr/supplierDetails/articles/1147534003
133
https://www2.hm.com/fr_fr/supplierDetails/articles/1135874003
134
https://www2.hm.com/fr_fr/supplierDetails/articles/1004271030
135
https://www2.hm.com/fr_fr/supplierDetails/articles/1173451001
136
https://www2.hm.com/fr_fr/supplierDetails/articles/1157325001
137
https://www2.hm.com/fr_fr/supplierDetails/articles/1124621007
138
https://www2.hm.com/fr_fr/supplierDetails/articles/1134231012
139
https://www2.hm.com/fr_fr/supplierDetails/articles/1066926005
140
https:

248
https://www2.hm.com/fr_fr/supplierDetails/articles/1160901001
249
https://www2.hm.com/fr_fr/supplierDetails/articles/1150326001
250
https://www2.hm.com/fr_fr/supplierDetails/articles/1132611005
251
https://www2.hm.com/fr_fr/supplierDetails/articles/1150604002
252
https://www2.hm.com/fr_fr/supplierDetails/articles/1142424002
253
https://www2.hm.com/fr_fr/supplierDetails/articles/1123078002
254
https://www2.hm.com/fr_fr/supplierDetails/articles/1122102004
255
https://www2.hm.com/fr_fr/supplierDetails/articles/1161939001
256
https://www2.hm.com/fr_fr/supplierDetails/articles/1158192001
257
https://www2.hm.com/fr_fr/supplierDetails/articles/1139036006
258
https://www2.hm.com/fr_fr/supplierDetails/articles/1135874004
259
https://www2.hm.com/fr_fr/supplierDetails/articles/1061711008
260
https://www2.hm.com/fr_fr/supplierDetails/articles/1136918001
261
https://www2.hm.com/fr_fr/supplierDetails/articles/1156066003
262
https://www2.hm.com/fr_fr/supplierDetails/articles/1159754002
263
https:

368
https://www2.hm.com/fr_fr/supplierDetails/articles/1162087001
369
https://www2.hm.com/fr_fr/supplierDetails/articles/1164669003
370
https://www2.hm.com/fr_fr/supplierDetails/articles/1176740005
371
https://www2.hm.com/fr_fr/supplierDetails/articles/1196462001
372
https://www2.hm.com/fr_fr/supplierDetails/articles/1159101001
373
https://www2.hm.com/fr_fr/supplierDetails/articles/1145299002
374
https://www2.hm.com/fr_fr/supplierDetails/articles/1166178004
375
https://www2.hm.com/fr_fr/supplierDetails/articles/1163972004
376
https://www2.hm.com/fr_fr/supplierDetails/articles/1132850002
377
https://www2.hm.com/fr_fr/supplierDetails/articles/1059293011
378
https://www2.hm.com/fr_fr/supplierDetails/articles/1146515004
379
https://www2.hm.com/fr_fr/supplierDetails/articles/1122105008
380
https://www2.hm.com/fr_fr/supplierDetails/articles/1166875004
381
https://www2.hm.com/fr_fr/supplierDetails/articles/1153251001
382
https://www2.hm.com/fr_fr/supplierDetails/articles/1135925001
383
https:

490
https://www2.hm.com/fr_fr/supplierDetails/articles/0776746035
491
https://www2.hm.com/fr_fr/supplierDetails/articles/1160495002
492
https://www2.hm.com/fr_fr/supplierDetails/articles/1129032003
493
https://www2.hm.com/fr_fr/supplierDetails/articles/1122069003
494
https://www2.hm.com/fr_fr/supplierDetails/articles/1182180002
495
https://www2.hm.com/fr_fr/supplierDetails/articles/1139237001
Error: Request failed with status code 400 for product ID: 1139237001
496
https://www2.hm.com/fr_fr/supplierDetails/articles/1139241002
Error: Request failed with status code 400 for product ID: 1139241002
497
https://www2.hm.com/fr_fr/supplierDetails/articles/1198400001
498
https://www2.hm.com/fr_fr/supplierDetails/articles/1175980001
499
https://www2.hm.com/fr_fr/supplierDetails/articles/1167592001
500
https://www2.hm.com/fr_fr/supplierDetails/articles/1168890001
501
https://www2.hm.com/fr_fr/supplierDetails/articles/1135356004
502
https://www2.hm.com/fr_fr/supplierDetails/articles/1155896002
50

613
https://www2.hm.com/fr_fr/supplierDetails/articles/1191556001
614
https://www2.hm.com/fr_fr/supplierDetails/articles/1147211003
615
https://www2.hm.com/fr_fr/supplierDetails/articles/1033197019
616
https://www2.hm.com/fr_fr/supplierDetails/articles/1132850001
617
https://www2.hm.com/fr_fr/supplierDetails/articles/1180352001
618
https://www2.hm.com/fr_fr/supplierDetails/articles/1121055013
619
https://www2.hm.com/fr_fr/supplierDetails/articles/1134305003
620
https://www2.hm.com/fr_fr/supplierDetails/articles/0875635036
621
https://www2.hm.com/fr_fr/supplierDetails/articles/1150850001
622
https://www2.hm.com/fr_fr/supplierDetails/articles/1121065011
623
https://www2.hm.com/fr_fr/supplierDetails/articles/1153520002
624
https://www2.hm.com/fr_fr/supplierDetails/articles/1176024001
625
https://www2.hm.com/fr_fr/supplierDetails/articles/1150536001
626
https://www2.hm.com/fr_fr/supplierDetails/articles/1180370001
627
https://www2.hm.com/fr_fr/supplierDetails/articles/1047224014
628
https:

734
https://www2.hm.com/fr_fr/supplierDetails/articles/1161535001
735
https://www2.hm.com/fr_fr/supplierDetails/articles/1182897001
736
https://www2.hm.com/fr_fr/supplierDetails/articles/1022138008
737
https://www2.hm.com/fr_fr/supplierDetails/articles/1082047009
738
https://www2.hm.com/fr_fr/supplierDetails/articles/1188391001
739
https://www2.hm.com/fr_fr/supplierDetails/articles/1115620003
740
https://www2.hm.com/fr_fr/supplierDetails/articles/1178300002
741
https://www2.hm.com/fr_fr/supplierDetails/articles/1121461002
742
https://www2.hm.com/fr_fr/supplierDetails/articles/1136317004
743
https://www2.hm.com/fr_fr/supplierDetails/articles/1155757002
744
https://www2.hm.com/fr_fr/supplierDetails/articles/1159343004
745
https://www2.hm.com/fr_fr/supplierDetails/articles/1146214001
746
https://www2.hm.com/fr_fr/supplierDetails/articles/1182228001
747
https://www2.hm.com/fr_fr/supplierDetails/articles/1170074009
748
https://www2.hm.com/fr_fr/supplierDetails/articles/1128562006
749
https:

858
https://www2.hm.com/fr_fr/supplierDetails/articles/1172126002
859
https://www2.hm.com/fr_fr/supplierDetails/articles/1110136002
860
https://www2.hm.com/fr_fr/supplierDetails/articles/1155996005
861
https://www2.hm.com/fr_fr/supplierDetails/articles/1159380003
862
https://www2.hm.com/fr_fr/supplierDetails/articles/1158701002
863
https://www2.hm.com/fr_fr/supplierDetails/articles/1094043006
864
https://www2.hm.com/fr_fr/supplierDetails/articles/1137522004
Error: Request failed with status code 400 for product ID: 1137522004
865
https://www2.hm.com/fr_fr/supplierDetails/articles/1192075001
866
https://www2.hm.com/fr_fr/supplierDetails/articles/1175630007
867
https://www2.hm.com/fr_fr/supplierDetails/articles/1137680014
868
https://www2.hm.com/fr_fr/supplierDetails/articles/1139520001
869
https://www2.hm.com/fr_fr/supplierDetails/articles/1151276001
870
https://www2.hm.com/fr_fr/supplierDetails/articles/1136411001
871
https://www2.hm.com/fr_fr/supplierDetails/articles/1149098004
872
ht

Error: Request failed with status code 400 for product ID: 1134949002
980
https://www2.hm.com/fr_fr/supplierDetails/articles/0956308039
981
https://www2.hm.com/fr_fr/supplierDetails/articles/0656677055
982
https://www2.hm.com/fr_fr/supplierDetails/articles/1157706002
983
https://www2.hm.com/fr_fr/supplierDetails/articles/1057589010
984
https://www2.hm.com/fr_fr/supplierDetails/articles/0971037008
985
https://www2.hm.com/fr_fr/supplierDetails/articles/1059431022
986
https://www2.hm.com/fr_fr/supplierDetails/articles/1035597009
987
https://www2.hm.com/fr_fr/supplierDetails/articles/0948441020
988
https://www2.hm.com/fr_fr/supplierDetails/articles/0973277043
989
https://www2.hm.com/fr_fr/supplierDetails/articles/1037593012
990
https://www2.hm.com/fr_fr/supplierDetails/articles/1150638001
Error: Request failed with status code 400 for product ID: 1150638001
991
https://www2.hm.com/fr_fr/supplierDetails/articles/1064750002
992
https://www2.hm.com/fr_fr/supplierDetails/articles/1194937001
99

In [230]:
# Créer un DataFrame pandas avec toutes les données récupérées
df_suplier = pd.DataFrame(all_rows)
df_suplier.to_csv("df_suplier.csv")
# Afficher le DataFrame
df_suplier.head()

Product ID Country                                   Company   
0  1166969003   Chine  SUZHOU LAOHONG KNITTING GARMENT CO.,LTD.  \
1  1142215002   Chine         Yunhan International (HK) Limited   
2  1159122003   Chine    SERENDIPITY INTERNATIONAL TRADING LTD.   
3  1159122001   Chine    SERENDIPITY INTERNATIONAL TRADING LTD.   
4  1157667001   Chine    SERENDIPITY INTERNATIONAL TRADING LTD.   

                                            Factory Number of Employees  
0  Shandong Wanli Fashion Co.,Ltd Jiu Nv Subsidiary            501-1000  
1                      Suzhou Xijin Fashion Co.,Ltd             101-500  
2                       Serendipity Fashion Co.,ltd             101-500  
3                       Serendipity Fashion Co.,ltd             101-500  
4                  Suhao Eastren Garments Co., Ltd.             101-500

Product ID Country                    Company            Factory   
460  1183077001   Maroc  VITA COUTURE COMPANY SARL  Vita Conf S.A.R.L  \

    Number of Employees  
460             101-500

array(['Shandong Wanli Fashion Co.,Ltd Jiu Nv Subsidiary',
       'Suzhou Xijin Fashion Co.,Ltd', 'Serendipity Fashion Co.,ltd',
       'Suhao Eastren Garments Co., Ltd.',
       'HANGZHOU JIAYI GARMENT COMPANY LTD',
       'Dongguan Dongsheng Shoes Ltd', 'CTA APPARELS (Unit D-235)',
       'CICEK KARDESLER TEKSTIL TUR. INS.NAK.GIDA SAN. VE TIC.LTD.STI.',
       'EGUM TEKSTIL SANAYI TIC LTD STI', 'Hubei Manly Fashion',
       'WANHE GARMENT COMPANY LIMITED', 'STERLING STYLES LTD',
       'Crystal Martin Apparel Bangladesh Ltd.',
       'CRYSTAL MARTIN (VIETNAM) CO.,LTD', 'PT. FAST MANUFACTURING',
       'SQ Birichina Ltd', 'Pak Chun Industrial Limited',
       'Winson Universal (Jiang Xi) Industry Ltd.',
       'Hop Yick ( Bangladesh) limited',
       'Suzhou Laohong knitting garment Co. , Ltd. NO.1 own fty',
       'Wuhu Seduno Fashion Co., Ltd.', 'KANIZ GARMENTS LIMITED',
       'NEWAGE APPARELS LIMITED', 'ARTISTIC APPARELS (PVT) LTD.',
       'ZHEJIANG HUADENG CLOTHING CO.,LTD.',
  